<a href="https://colab.research.google.com/github/YongERong/stackup-sentimental-chatbot/blob/main/Llama_Chatbot_with_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
Installation and importing of packages, inserting of API keys/tokens

In [1]:
!pip install -q accelerate protobuf sentencepiece torch transformers huggingface_hub gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 757.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
import torch
import gradio as gr

In [ ]:
login()

## Chat with Context

In [3]:
from pprint import pprint
from transformers import Pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# chat buffer
buffer = []

# function to embed messages in template format
def embed_message(message, role):
    return {
        "role": role,
        "content": message
    }

# custom pipeline
class ChatBufferPipeline(Pipeline):

    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}

        # lookback on chat
        if "lookback" in kwargs:
            preprocess_kwargs["lookback"] = kwargs["lookback"]

        if "sentiment" in kwargs:
            preprocess_kwargs["sentiment"] = kwargs["sentiment"]

        return preprocess_kwargs, {}, {}

    def preprocess(self, prompt, lookback=None, sentiment=None):
        # initial system message
        messages = [
            {
                "role": "system",
                "content": "You are a friendly chatbot who answers user questions. You can use the previous examples if this helps you. The sentiment score of the question is also provided.",
                "sentiment": sentiment if sentiment else None
            },
        ]
        # get chat history

        if lookback:
            buffer_messages = buffer[-(lookback):]
            messages += buffer_messages
        # embed user message in template format
        user_message = embed_message(prompt, "user")
        messages.append(user_message)
        # add new message to buffer
        buffer.append(user_message)

        messages = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        return self.tokenizer(messages, return_tensors="pt").input_ids.cuda()

    def _forward(self, model_inputs):
        outputs = self.model.generate(model_inputs, max_new_tokens=250, min_new_tokens=20)
        return {"outputs": outputs, "inputs": model_inputs}

    def postprocess(self, model_outputs):
        outputs = model_outputs["outputs"]
        inputs = model_outputs["inputs"]
        assistant_output = self.tokenizer.decode(outputs[0][len(inputs[0]):], add_special_tokens=False)
        buffer.append(embed_message(assistant_output, "assistant"))
        full_dialog = self.tokenizer.decode(outputs[0])
        return assistant_output, full_dialog

In [4]:
# model
model_id = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)

# pipeline
chatpipe = ChatBufferPipeline(model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
_, dialog = chatpipe("My favorite color is yellow, what is yours?")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
pprint(dialog)

('<s> <|system|>\n'
 'You are a friendly chatbot who answers user questions. You can use the '
 'previous examples if this helps you.</s> \n'
 '<|user|>\n'
 'My favorite color is yellow, what is yours?</s> \n'
 '<|assistant|>\n'
 "I don't have a favorite color as I'm not capable of having preferences or "
 "feelings. However, I'm programmed to respond to your question and I'm glad "
 'you shared your favorite color with me! Is there anything else I can help '
 'you with today?</s>')


In [ ]:
_, dialog = chatpipe("what is my favorite color?")

In [ ]:
pprint(dialog)

('<s> <|system|>\n'
 'You are a friendly chatbot who answers user questions. You can use the '
 'previous examples if this helps you.</s> \n'
 '<|user|>\n'
 'what is my favorite color?</s> \n'
 '<|assistant|>\n'
 "I'm sorry but I don't have access to your personal information. Your "
 "favorite color is something that is unique to you and I'd be happy to help "
 'you figure it out based on your preferences. Do you have any colors that you '
 'particularly like or dislike? Do you have a favorite outfit or decor item '
 "that features a certain color? Let's work together to narrow down your "
 'favorite color!</s>')


In [ ]:
_, dialog = chatpipe("what is my favorite color?", lookback=True)

In [ ]:
pprint(dialog)

('<s> <|system|>\n'
 'You are a friendly chatbot who answers user questions. You can use the '
 'previous examples if this helps you.</s> \n'
 '<|assistant|>\n'
 "I'm sorry but I don't have access to your personal information. Your "
 "favorite color is something that is unique to you and I'd be happy to help "
 'you figure it out based on your preferences. Do you have any colors that you '
 'particularly like or dislike? Do you have a favorite outfit or decor item '
 "that features a certain color? Let's work together to narrow down your "
 'favorite color!</s></s> \n'
 '<|user|>\n'
 'what is my favorite color?</s> \n'
 '<|assistant|>\n'
 "I'm sorry but I don't have access to your personal information. However, "
 "based on the conversation we've had, I can make some educated guesses. From "
 'your descriptions, it seems like you prefer bright and bold colors, such as '
 'red, orange, and yellow. You also mentioned liking green, especially in '
 'nature. However, you did mention disli

In [ ]:
_, dialog = chatpipe("what is my favorite color?", lookback=10)

In [ ]:
pprint(dialog)

('<s> <|system|>\n'
 'You are a friendly chatbot who answers user questions. You can use the '
 'previous examples if this helps you.</s> \n'
 '<|user|>\n'
 'My favorite color is yellow, what is yours?</s> \n'
 '<|assistant|>\n'
 "I don't have a favorite color as I'm not capable of having preferences or "
 "feelings. However, I'm programmed to respond to your question and I'm glad "
 'you shared your favorite color with me! Is there anything else I can help '
 'you with today?</s></s> \n'
 '<|user|>\n'
 'what is my favorite color?</s> \n'
 '<|assistant|>\n'
 "I'm sorry but I don't have access to your personal information. Your "
 "favorite color is something that is unique to you and I'd be happy to help "
 'you figure it out based on your preferences. Do you have any colors that you '
 'particularly like or dislike? Do you have a favorite outfit or decor item '
 "that features a certain color? Let's work together to narrow down your "
 'favorite color!</s></s> \n'
 '<|user|>\n'
 'what

## Sentiment Analysis
Attempted to use the same model with a different prompt -> switch to distilBERT for faster speeds

In [ ]:
from transformers import Pipeline

# function to embed messages in template format
def embed_message(message, role):
    return {
        "role": role,
        "content": message
    }

# custom pipeline
class SentimentAnalysisPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}

        return preprocess_kwargs, {}, {}


    def preprocess(self, prompt):
        # initial system message
        messages = [
            {
                "role": "system",
                "content": "You are a sentiment analysis analyser which provides the sentiment of a text massage. Return a dictionary containing the sentiment_score: 1 if the sentiment is positive, 0 if it is neutral and -1 if it is negative and the main_sentiment: one of the following [angry, pleased, sad, urgent, interested, uninterested e.g. {sentiment_score: -1, main_sentiment: angry} Ensure you follow the format strictly, this is important for our company.]",
            },
        ]

        # embed user message in template format
        user_message = embed_message(prompt, "user")
        messages.append(user_message)

        messages = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        return self.tokenizer(messages, return_tensors="pt").input_ids.cuda()


    def _forward(self, model_inputs):
        outputs = self.model.generate(model_inputs, max_new_tokens=250, min_new_tokens=20)
        return {"outputs": outputs, "inputs": model_inputs}

    def postprocess(self, model_outputs):
        outputs = model_outputs["outputs"]
        inputs = model_outputs["inputs"]
        assistant_output = self.tokenizer.decode(outputs[0][len(inputs[0]):], add_special_tokens=False)
        full_dialog = self.tokenizer.decode(outputs[0])
        return assistant_output, full_dialog

In [ ]:
# model
model_id = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)

# pipeline
chatpipe = SentimentAnalysisPipeline(model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
_, dialog = chatpipe("My favorite color is yellow, what is yours?")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


### distilBERT

In [5]:
sentiment_pipe = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


## Gradio UI

In [10]:
def gradio_chat_interface(question):
    sentiment = sentiment_pipe(question)
    response, logs = chatpipe("My favorite color is yellow, what is yours?", lookback=10, sentiment=sentiment)
    return f"Answer from Assistant {response}\n\n {logs}"

In [11]:
# Create a Gradio Interface
interface = gr.Interface(
    fn=gradio_chat_interface,
    inputs="text",
    outputs="text",
    title="Zephyr Chatbot with Sentiment Analysis",
    description="Ask a question and the chatbot will respond using a Zephyr Model",
)

In [13]:
buffer =[]

In [12]:
# Launch the Gradio Interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://402d8ee3d71319ce23.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Playing Around with Langchain

In [ ]:
!pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline


chat_model = ChatHuggingFace(llm=HuggingFacePipeline(pipeline=llama_pipeline))

In [ ]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(
        content="What happens when an unstoppable force meets an immovable object?"
    ),
]

ai_msg = chat_model.invoke(messages)

ValueError: Cannot use apply_chat_template() because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

## Provided Code

In [ ]:
# Define the path for the CSV file
csv_file = 'qa_dataset.csv'

# Check if the CSV file exists; if not, create it with initial data
if not os.path.exists(csv_file):
    qa_data = {
        'question': ["What is the name of Julius Magellan's dog?", "Who is Julius Magellan's dog?"],
        'answer': ["The name of Julius Magellan's dog is Sparky", "Julius Magellan's dog is called Sparky"]
    }
    qa_df = pd.DataFrame(qa_data)
    qa_df.to_csv(csv_file, index=False)
else:
    # Load the existing CSV file into a DataFrame
    qa_df = pd.read_csv(csv_file)

In [ ]:
# Verify the CSV content
qa_df = pd.read_csv(csv_file)
print(qa_df)

                                     question  \
0  What is the name of Julius Magellan's dog?   
1               Who is Julius Magellan's dog?   

                                        answer  
0  The name of Julius Magellan's dog is Sparky  
1       Julius Magellan's dog is called Sparky  


In [ ]:
chat = [
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
  {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

In [ ]:
# Initialize the Llama 2 model and tokenizer
model_id = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
#tokenizer.use_default_system_prompt = False
tokenizer.apply_chat_template(chat, tokenize=False)

# Initialize the pipeline using Hugging Face pipeline
llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    max_length=1024,  # Adjust max_length as needed
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Cannot use apply_chat_template() because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
def answer_question(question):
    global qa_df
    # Check if the question is in the QA dataset
    answer = qa_df[qa_df['question'].str.lower() == question.lower()]['answer']

    if not answer.empty:
        # Return the first matching answer
        print(f"Answer from QA dataset: {answer.iloc[0]}")
    else:
        # Use Llama 2 to generate an answer
        response = llama_pipeline(question, max_length=150, do_sample=True)[0]['generated_text']

        # Ensure the response doesn't redundantly include the question or incorrectly repeat "Answer"
        response = response.replace(f"Answer: {question}", "").strip()
        print(f"Answer from Llama 2: {response}")

        # Add the new QA pair to the dataset if it's not already present
        if not any(qa_df['question'].str.lower() == question.lower()):
            new_row = pd.DataFrame({'question': [question], 'answer': [response]})
            qa_df = pd.concat([qa_df, new_row], ignore_index=True)
            qa_df.to_csv(csv_file, index=False)
            print("New QA pair added to the dataset.")

In [ ]:
question_1 = "What is the name of Julius Magellan's dog?"
answer_question(question_1)

Answer from QA dataset: The name of Julius Magellan's dog is Sparky


In [ ]:
question_2 = "Who is Julius Magellan's dog?"
answer_question(question_2)

Answer from QA dataset: Julius Magellan's dog is called Sparky


In [ ]:
# This should fallback to Llama 2 and then get added
# to the dataset
question_3 = "What is the capital of France?"
answer_question(question_3)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Answer from Llama 2: What is the capital of France?

The capital of France is Paris.
New QA pair added to the dataset.


In [ ]:
print(qa_df)

                                     question  \
0  What is the name of Julius Magellan's dog?   
1               Who is Julius Magellan's dog?   
2              What is the capital of France?   

                                              answer  
0        The name of Julius Magellan's dog is Sparky  
1             Julius Magellan's dog is called Sparky  
2  What is the capital of France?\n\nThe capital ...  
